In [1]:
import numpy as np
import matplotlib.pyplot as plt

from qutip import *

In [ ]:
# Helper functions

def check_trace(rho):
    """
    Check if the trace of the density matrix is 1.
    """
    return np.isclose(rho.tr(), 1.0, atol=1e-10)

def check_hermitian(H):
    """
    Check if the operator is Hermitian.
    """
    return np.allclose(H, H.dag(), atol=1e-10)

def check_psd(mat):
    """
    Check if the matrix is positive semi-definite.
    """
    return np.all(np.linalg.eigvals(mat) >= -1e-10)


In [ ]:
def check_choi_decomposition(choi_matrix, pos_choi, neg_choi):
    """
    Check if the Choi matrix is decomposed correctly.
    """
    return np.all(choi_matrix - pos_choi + neg_choi >= -1e-10)

In [ ]:
def construct_choi_matrix(map):

    # TODO: verify if this is the correct way to construct the Liouvillian superoperator, currently generated by copilot 


    """
    Construct the Choi matrix for the map.
    """
    dim = map.shape[0]
    choi_matrix = np.zeros((dim**2, dim**2), dtype=complex)
    
    for i in range(dim):
        for j in range(dim):
            for k in range(dim):
                for l in range(dim):
                    choi_matrix[i*dim + j, k*dim + l] = map[i, k] * np.conjugate(map[j, l])
    
    return choi_matrix


def positive_and_negative_eigenspaces(choi_matrix):
    """
    Compute the positive and negative eigenspaces of the Choi matrix.
    """
    eigenvalues, eigenvectors = np.linalg.eig(choi_matrix)
    
    # Positive eigenspace
    pos_eigenvalues = eigenvalues[eigenvalues > 0]
    pos_eigenvectors = eigenvectors[:, eigenvalues > 0]
    
    # Negative eigenspace
    neg_eigenvalues = eigenvalues[eigenvalues < 0]
    neg_eigenvectors = eigenvectors[:, eigenvalues < 0]
    
    return pos_eigenvalues, pos_eigenvectors, neg_eigenvalues, neg_eigenvectors

def construct_CP_map_from_eigenspace(eigenvalues, eigenvectors):
    """
    Construct the map from the positive and negative eigenspaces.
    """
    dim = eigenvectors.shape[0]
    CP_map = np.zeros((dim, dim), dtype=complex)
    
    for i in range(len(eigenvalues)):
        CP_map += eigenvalues[i] * np.outer(eigenvectors[:, i], eigenvectors[:, i].conjugate())
    
    return CP_map


def decompose_into_extremal_CP_maps(choi_matrix_cp_map):
    
    pass

In [ ]:
# Lindbladian to General Dynamical Map

def get_liouvillian(H, l_ops):

    # TODO: verify if this is the correct way to construct the Liouvillian superoperator, currently generated by copilot 


    """
    Get the Liouvillian superoperator from the Hamiltonian and Lindblad operators.
    """
    # Construct the Liouvillian superoperator
    L = -1j * (tensor(H, qeye(2)) - tensor(qeye(2), H)).tr()
    
    for l_op in l_ops:
        L += (tensor(l_op.dag(), qeye(2)) @ tensor(qeye(2), l_op) - 0.5 * (tensor(l_op.dag() * l_op, qeye(2)) + tensor(qeye(2), l_op.dag() * l_op))).tr()
    
    return L


def get_dynamical_map(L, dt):
    """
    Get the dynamical map from the Liouvillian superoperator for time dt.
    """
    # Time evolution operator
    U = (-1j * L * dt).expm()  
    
    return U

